# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(len(file_path_list), "files present")

/home/workspace
30 files present


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)



#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)



### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1:

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

table1_query = "CREATE TABLE IF NOT EXISTS song_session "
table1_query += "(sessionId int, itemInSession int, artist text, song_title text, song_length FLOAT, "
table1_query += "PRIMARY KEY (sessionId, itemInSession)"
table1_query += ")"    

try:
    session.execute(table1_query)
except Exception as e:
    print(e)
    
    
    
# defines select query statement for table1
query1 = """
SELECT artist, song_title, song_length from song_session
WHERE sessionID=%s 
AND itemInSession=%s
"""


In [9]:
# Extract headers and its indices
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    for header in csvreader:
        header_names = header
        break
       
header_lookup = {  v:i  for i,v in enumerate(header_names) }
header_lookup

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_session(sessionId, itemInSession, artist, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)" # 5 items
 
        # some `line` elements must be cast from the default 'string' format to appropriate types according to defined types
    
        session.execute(query, ( int(line[header_lookup['sessionId']]), int(line[header_lookup['itemInSession']]), 
                                line[header_lookup['artist']],line[header_lookup['song']],float(line[header_lookup['length']])) )

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

try:
    rows = session.execute(query1, (338, 4))
except Exception as e:
    print(e)

colnames=["artist","song_title","song_length"]
query1_df = pd.DataFrame(list(rows),columns=colnames)
query1_df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### QUERY 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

In [12]:
table2_query = """
CREATE TABLE IF NOT EXISTS artist_session 
( 
    artist text, 
    song_title text, 
    user_first_name text, 
    user_last_name text, 
    itemInSession int, 
    userId int, 
    sessionId int
    
, PRIMARY KEY (userId, sessionId, itemInSession)
)

"""
    
try:
    session.execute(table2_query)
except Exception as e:
    print(e)
    
    
query2 = """
SELECT artist, song_title, user_first_name, user_last_name FROM artist_session
                    WHERE userId = %s
                    AND sessionId = %s 

"""

In [13]:
header_lookup

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_session(artist, song_title, user_first_name, user_last_name, itemInSession, userId, sessionId )"
        query += " VALUES ("
        query += "%s,"*6 + "%s)" # inserting 7 fields
        
        session.execute(query,  ( 
            line[header_lookup['artist']],
            line[header_lookup['song']],
            line[header_lookup['firstName']],
            line[header_lookup['lastName']],
            
            int(line[header_lookup['itemInSession']]),
            int(line[header_lookup['userId']]),
            int(line[header_lookup['sessionId']])
            
        ))

In [15]:
## SELECT statement for Table 2
try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)

colnames=['artist', 'song_title', 'user_first_name', 'user_last_name' ]
query2_df = pd.DataFrame(list(rows),columns=colnames)
query2_df


,artist,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### QUERY 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

table3_query = """
CREATE TABLE IF NOT EXISTS user_session 
(
user_first_name text,
user_last_name text,
song text,
userId int,

PRIMARY KEY (song, userId) 
)
"""

# `song` as parition, userId as clustering column
try:
    session.execute(table3_query)
except Exception as e:
    print(e)  
    
    
query3 = """
SELECT user_first_name, user_last_name FROM user_session
WHERE song=%s; 
"""

In [17]:
header_lookup

{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

In [18]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query= "INSERT INTO user_session (user_first_name, user_last_name, song, userId) "
        query+= "VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (
            line[header_lookup['firstName']],
            line[header_lookup['lastName']],   
            line[header_lookup['song']], 
            int(line[header_lookup['userId']]), 
        ))
        

In [19]:
try:
    rows = session.execute(query3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

colnames=['user_first_name', 'user_last_name']
query3_df = pd.DataFrame(list(rows),columns=colnames)
query3_df

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [20]:
session.execute("DROP TABLE IF EXISTS song_session")
session.execute("DROP TABLE IF EXISTS artist_session")
session.execute("DROP TABLE IF EXISTS user_session")

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()